In [1]:
%useLatestDescriptors
%use coroutines

@file:DependsOn("dev.langchain4j:langchain4j:1.0.0-beta2")
@file:DependsOn("dev.langchain4j:langchain4j-anthropic:1.0.0-beta2")

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY")

In [3]:
import dev.langchain4j.data.message.SystemMessage.systemMessage
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.model.anthropic.AnthropicChatModel
import dev.langchain4j.model.anthropic.AnthropicChatModelName
import dev.langchain4j.model.chat.chat

suspend fun llmCall(
    prompt: String,
    systemPrompt: String? = null,
    model: AnthropicChatModelName = AnthropicChatModelName.CLAUDE_3_7_SONNET_20250219
): String {
    val client = AnthropicChatModel.builder()
        .apiKey(apiKey)
        .modelName(model)
        .maxTokens(4096)
        .temperature(0.1)
        .build()

    return withContext(Dispatchers.IO) {
        val response = client.chat {
            systemPrompt?.let { messages += systemMessage(it) }
            messages += userMessage(prompt)
        }
        response.aiMessage().text()
    }
}

In [4]:
val prompts = listOf(
                                // Step 1
    """
    Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth
    """,
                                // Step 2
    """
    Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth
    """,
                                // Step 3
    """
    Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction
    """,
                                // Step 4
    """
    Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |
    """
)

In [5]:
val report =
    """
    Q3 Performance Summary:
    Our customer satisfaction score rose to 92 points this quarter.
    Revenue grew by 45% compared to last year.
    Market share is now at 23% in our primary market.
    Customer churn decreased to 5% from 8%.
    New user acquisition cost is $43 per user.
    Product adoption rate increased to 78%.
    Employee satisfaction is at 87 points.
    Operating margin improved to 34%.
    """.trimIndent()

In [6]:
suspend fun chain(input: String, prompts: List<String>): String {
    var result = input
    prompts.forEachIndexed { index, prompt ->
        println("Step ${index + 1}")
        result = llmCall("$prompt\nInput: $result")
        println(result)
    }
    return result
}

In [7]:
runBlocking {
    println("Input text:")
    println(report)
    val formattedResult = chain(report, prompts)
    println(formattedResult)
}

Input text:
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
Step 1
92: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
$43: new user acquisition cost
78%: product adoption rate
87: employee satisfaction
34%: operating margin
Step 2
92%: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
$43: new user acquisition cost
78%: product adoption rate
87%: employee satisfaction
34%: operating margin
Step 3
92%: customer satisfaction score
87%: employee satisfaction
78%: product adoption rate
45%: revenue growth
34%: operating margin
23%: market share
5%: customer churn
$43: new user acquisition cost
S